In [1]:
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)

import numpy as np
import pandas as pd
from copy import deepcopy
from Merge import merge
from copy import deepcopy
from Logic import logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, logic_series_print
from Battery import Battery
from Predictions import RF
import ast
from DPModel import DP_P2P

In [2]:
def get_hyperparams(house, t, date):
    hp = pd.read_csv('hyperparams.csv')
    c1 = hp['House'] == house
    c2 = hp['Type'] == t
    c3 = hp['Date'] == date
    return hp.loc[c1 & c2 & c3, 'opt_size'].values[0], ast.literal_eval(hp.loc[c1 & c2 & c3, 'opt_lags'].values[0]),eval(hp.loc[c1 & c2 & c3, 'opt_params'].values[0])

In [3]:
Start = '2022-01-08 00:00:00'
End = '2022-01-08 23:00:00'
month = ('Summer' if pd.to_datetime(Start).month==7 else 'Winter')
N = len(pd.date_range(start=Start,end=End,freq="h"))

bat = Battery(max_capacity=13,max_charge=7)
max_number_states=20

houses = ["h16","h28","h22",'h32']
merges = [merge(house) for house in houses]
x0 = (0,0,0,0)

rf_models = [RF(house) for house in houses]

for i,house in enumerate(houses):
    Popt_size, Popt_lag, Popt_params = get_hyperparams(house, 'Production', month)
    Copt_size, Copt_lag, Copt_params = get_hyperparams(house, 'Consumption', month)

    prodforc = rf_models[i].get_forecaster(Popt_lag,Popt_params)  
    consforc = rf_models[i].get_forecaster(Copt_lag,Copt_params)

    rf_models[i].set_forecaster("p",Popt_size,prodforc)
    rf_models[i].set_forecaster("c",Copt_size,consforc)

stepsize = 2

In [4]:
num_loops = int(np.ceil(N/stepsize))
remainder = N%stepsize
length = stepsize
Start_i = Start
x0_i = x0

series = [pd.DataFrame() for _ in range(len(houses))]

for i in range(num_loops):
    if i == num_loops-1:
        length = length if remainder == 0 else remainder

    End_i = pd.date_range(start=Start_i, periods=24, freq="h")[-1]
    
    print(f"Loop number {i+1} of {num_loops}")
    print(f"Predicting period from {Start_i} to {End_i} (n = {len(pd.date_range(Start_i, End_i, freq='H'))})")
    
    preds_i = [rf.get_predictions(Start_i, End_i, get_carb = j==0) for j,rf in enumerate(rf_models)]
    
    for j in range(len(preds_i)-1):
        preds_i[j+1]["CO2Emission"] = preds_i[0]["CO2Emission"]
        
    print()
    print(f"Taking actions the first {length} actions from {Start_i} to {preds_i[0].index[length-1]}")
    
    dpp2p = DP_P2P(Start_i,End_i,preds_i,houses,deepcopy(bat))
    dpp2p.P2P_sol(x0_i, max_number_states=20, byday=True, verbose=True, acts=False)
    dpp2p.merges = merges
    dpp2p.ep = rf_models[0].get_emissions(Start_i, End_i)/1000
    dpp2p.constraints_ord = None
    dpp2p.P2P_sol(x0_i, max_number_states=20, byday=True, verbose=True, acts=True)
    
    series_i = dpp2p.all_series
    
    for j in range(len(series_i)):
        series_i[j] = series_i[j].iloc[:length, ]
    
    Start_i = pd.date_range(start=Start_i,periods=length+1,freq="h")[-1]
    
    temp = list(x0_i)
    for j in range(len(series_i)):
        temp[j] = series_i[j]["capacity_after"].iloc[length-1]
        
    x0_i = tuple(temp)
        
    for j in range(len(houses)):
        series[j] = pd.concat([series[j],series_i[j]])
    
    
    results_dict = {house: series[j] for j, house in enumerate(houses)}
    print()
    print("Done!")
    print()
    print()
    
for j in range(len(houses)):
    series[j]["cost_cummulative"] = series[j]["cost"].cumsum(axis=0)
    series[j]["emission_cummulative"] = series[j]["emission"].cumsum(axis=0)
    

Loop number 1 of 12
Predicting period from 2022-01-08 00:00:00 to 2022-01-08 23:00:00 (n = 24)

Taking actions the first 2 actions from 2022-01-08 00:00:00 to 2022-01-08 01:00:00


KeyboardInterrupt: 

In [ ]:
series[0].to_csv('Results_DP\h16_cluster2_winter.csv')
series[1].to_csv('Results_DP\h28_cluster2_winter.csv')
series[2].to_csv('Results_DP\h22_cluster2_winter.csv')
series[3].to_csv('Results_DP\h32_cluster2_winter.csv')

In [20]:
results_dict['h28']

,prod_h28,cons_h28,yield,SpotPriceDKK,CO2Emission,capacity_before,capacity_degraded,capacity_after,surplus,charge,buy,cost,emission,cost_cummulative,emission_cummulative,grid,peer
2022-07-02 00:00:00,0.0,4.0,-4.0,2125.239990,175.0,0.0,0.0,0.0,-4.0,-0.0,0.0,11.552960,0.7000,11.552960,0.7000,4.0,0.0
2022-07-02 01:00:00,0.0,3.0,-3.0,1896.739990,169.0,0.0,0.0,0.0,-3.0,-0.0,0.0,7.979220,0.5070,19.532180,1.2070,3.0,0.0
2022-07-02 02:00:00,0.0,2.9,-2.9,1678.880005,139.0,0.0,0.0,0.4,-3.3,0.4,0.4,8.058204,0.4587,27.590384,1.6657,3.3,0.0
2022-07-02 03:00:00,0.0,3.0,-3.0,1716.069946,119.0,0.4,0.4,0.0,-2.6,-0.4,0.0,4.461782,0.0000,32.052166,1.6657,0.0,2.6
2022-07-02 04:00:00,0.0,2.9,-2.9,1650.160034,189.0,0.0,0.0,0.0,-2.9,-0.0,0.0,6.998164,0.5481,39.050330,2.2138,2.9,0.0
2022-07-02 05:00:00,0.0,2.8,-2.8,1593.260010,184.0,0.0,0.0,0.4,-3.2,0.4,0.4,7.540032,0.5888,46.590362,2.8026,3.2,0.0
2022-07-02 06:00:00,0.3,2.5,-2.2,1739.349976,193.0,0.4,0.4,0.0,-1.8,-0.4,0.0,4.275330,0.2895,50.865692,3.0921,1.5,0.3
2022-07-02 07:00:00,0.5,2.4,-1.9,1819.089966,212.0,0.0,0.0,0.0,-1.9,-0.0,0.0,4.829671,0.3816,55.695363,3.4737,1.8,0.1
2022-07-02 08:00:00,1.4,2.0,-0.6,1551.609985,177.0,0.0,0.0,0.0,-0.6,-0.0,0.0,1.388766,0.1062,57.084129,3.5799,0.6,0.0
2022-07-02 09:00:00,4.6,1.7,2.9,1146.079956,113.0,0.0,0.0,0.0,2.9,-0.0,0.0,-3.323632,-0.3277,53.760497,3.2522,-2.9,0.0


In [14]:
h16